#Set-Up

In [1]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo. 
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
#!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

     |████████████████████████████████| 9.7 MB 5.5 MB/s 
     |████████████████████████████████| 49 kB 2.7 MB/s 


In [ ]:
import pandas as pd

#Your Turn Looping Solution

Find the minimum of the function $f(x) = ax^2 - bx + c$

For the 100 problems with different instances of a, b, c, given in the dataset.

Below, I've started the problem by solving just one instance.

In [7]:
#load in data
import pandas as pd
df = pd.read_excel("YTLoopData.xlsx", sheet_name = "data")
df

,Problem_Number,a,b,c
0,1,4,15,34
1,2,2,9,10
2,3,1,12,33
3,4,5,3,64
4,5,4,6,19
...,...,...,...,...
95,96,5,5,78
96,97,3,15,73
97,98,3,5,100
98,99,5,10,57


In [ ]:
a = 4
b = 15
c = 34

#initialize a "Concrete Model"
model = ConcreteModel()

#initialize DVs
model.x = Var(domain = Reals) #You can google "pyomo domain sets" to see several domain examples

#define the objective
model.Objective = Objective(expr = a*model.x**2 - b*model.x + c, sense = minimize)

#(Optional) You can use model.pprint() to see what you've done so far
model.pprint()

In [ ]:
#solve model
opt = SolverFactory('ipopt', executable='/content/ipopt')

results = opt.solve(model, tee = True) #setting tee = False hides the diagnostic outputs

In [10]:
#print relevant values
print("x* = ", model.x()) #alternatively you can use value(model.x)
print("obj* = ", model.Objective()) #alternatively you can use value(model.Objective)

x* =  1.875
obj* =  19.9375


In [11]:
# convert to fucntion
def solveproblem(a,b,c):
  #initialize a "Concrete Model"
  model = ConcreteModel()
  #initialize DVs
  model.x = Var(domain = Reals) #You can google "pyomo domain sets" to see several domain examples
  #define the objective
  model.Objective = Objective(expr = a*model.x**2 - b*model.x + c, sense = minimize)
  #(Optional) You can use model.pprint() to see what you've done so far
  #model.pprint()
  #solve model
  opt = SolverFactory('ipopt', executable='/content/ipopt')
  results = opt.solve(model, tee = True)

  return{"x0*":model.x(), "obj*":model.Objective()}
                      

In [12]:
row_data = df.iloc[0].values.tolist()
row_data

[1, 4, 15, 34]

In [15]:
a = row_data[1]
b = row_data[2]
c = row_data[3]

34

In [ ]:
solutions = []
for i in range(len(df)):
  row_data = df.iloc[i].values.tolist()
  a = row_data[1]
  b = row_data[2]
  c = row_data[3]
  solutions.append(solveproblem(a,b,c))

In [17]:
df["solutions"] = solutions
df

,Problem_Number,a,b,c,solutions
0,1,4,15,34,"{'x0*': 1.875, 'obj*': 19.9375}"
1,2,2,9,10,"{'x0*': 2.25, 'obj*': -0.125}"
2,3,1,12,33,"{'x0*': 6.0, 'obj*': -3.0}"
3,4,5,3,64,"{'x0*': 0.3, 'obj*': 63.55}"
4,5,4,6,19,"{'x0*': 0.75, 'obj*': 16.75}"
...,...,...,...,...,...
95,96,5,5,78,"{'x0*': 0.5, 'obj*': 76.75}"
96,97,3,15,73,"{'x0*': 2.5, 'obj*': 54.25}"
97,98,3,5,100,"{'x0*': 0.8333333333333333, 'obj*': 97.9166666..."
98,99,5,10,57,"{'x0*': 1.0, 'obj*': 52.0}"
